In [1]:
import pandas as pd
train= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/NIKL_SKT_WiC_Train.tsv", delimiter= '\t')
test= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/국립국어원/NIKL_SKT_WiC_Test.tsv", delimiter= '\t')
test.head()

,ID,Target,SENTENCE1,SENTENCE2,ANSWER,start_s1,end_s1,start_s2,end_s2
0,1,양분,토양에 양분이 풍부하여 나무가 잘 자란다.,태아는 모체로부터 양분과 산소를 공급받게 된다.,NaN,4,6,10,12
1,2,철,철에 따라 피는 꽃.,나는 손에 묻은 분필 가루를 털면서 철 늦은 오버를 치렁치렁 걸치고 걷는 교수님의 ...,NaN,0,1,20,21
2,3,풍경,처마 끝에서 풍경이 울다.,방 안 풍경을 둘러보다.,NaN,7,9,4,6
3,4,경사,완만한 경사를 이룬 언덕.,그 산은 경사가 급해서 오르기가 힘들다.,NaN,4,6,5,7
4,5,시,시 한 편을 낭송하다.,규칙을 어겼을 시에는 처벌을 받는다.,NaN,0,1,8,9


In [2]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")

model = AutoModelWithLMHead.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:592: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [4]:
import torch

In [30]:
#######################gpt2 생성해보기 예제

input_ids = tokenizer.encode("신기한데? 생각보다")
input_ids


[36645, 35943, 407, 39272]

In [31]:
 gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=20,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)


신기한데? 생각보다 훨씬 더 맛있었어요! 
 그리고 또 하나!! 바로바로 이 팥


In [15]:
train_SENTENCE1_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.SENTENCE1]
train_SENTENCE2_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.SENTENCE2]

train_SENTENCE1_bert[:5]

['[CLS] 그의 죽음은 타살로 단정이 되었다. [SEP]',
 '[CLS] 현대 생활에서 단전과 단수의 고통은 겪어 보지 않으면 짐작도 못한다. [SEP]',
 '[CLS] 화성은 밤과 낮, 하루의 길이와 계절의 변화가 지구와 매우 비슷하다. [SEP]',
 '[CLS] 달의 자전 주기는 달이 지구의 둘레를 공전하는 주기와 같다. [SEP]',
 '[CLS] 오늘의 적이 내일은 동지가 될 수 있다. [SEP]']

In [18]:
tokenized_train_SENTENCE1 = [tokenizer.tokenize(s) for s in train.SENTENCE1]
print(tokenized_train_SENTENCE1[0])

['▁그의', '▁죽음', '은', '▁타', '살', '로', '▁단', '정이', '▁되었다.']


In [19]:
tokenized_train_SENTENCE2 = [tokenizer.tokenize(s) for s in train.SENTENCE2]
tokenized_test_SENTENCE1=[tokenizer.tokenize(s) for s in test.SENTENCE1]
tokenized_test_SENTENCE2= [tokenizer.tokenize(s) for s in test.SENTENCE2]

In [33]:
!pip install sentencepiece


     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [37]:
##################################t5
  
from transformers import T5Tokenizer, T5ForConditionalGeneration
from dataset import NIKL_SKT_WiC, T5_Classification_Collator
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import random
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

random_seed = 42

random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("Load T5 model...")
tokenizer = T5Tokenizer.from_pretrained("KETI-AIR/ke-t5-large")
model = T5ForConditionalGeneration.from_pretrained("KETI-AIR/ke-t5-large")
model.cuda()

batch_size = 1

collator = T5_Classification_Collator(use_tokenizer=tokenizer, max_sequence_len=128)

print("Load Dataset...")
train_dataset = NIKL_SKT_WiC('NIKL_SKT_WiC_Train.tsv')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator)

dev_dataset = NIKL_SKT_WiC('NIKL_SKT_WiC_Dev.tsv')
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)

epochs = 10
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = len(train_dataloader), # Default value in run_glue.py
                                            num_training_steps = total_steps)

print("Training...")

for epoch_i in range(epochs):
  print("-------------------------------------------------------------")
  # train
  model.train()

  true_labels = []
  prediction_labels = []
  total_loss = 0

  for batch in tqdm(train_dataloader):
    true_labels += batch['labels'].numpy().flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    model.zero_grad()
    outputs = model(**batch)

    loss, logits = outputs[:2]
    total_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

    logits = logits.detach().cpu().numpy()

    prediction_labels += logits.argmax(axis=-1).flatten().tolist()

  train_avg_epoch_loss = total_loss / len(train_dataloader)
  train_acc = accuracy_score(true_labels, prediction_labels)

  prediction_labels = []
  true_labels = []
  total_loss = 0

  model.eval()

  # Evaluate data for one epoch
  for batch in tqdm(dev_dataloader):
    true_labels += (batch['labels'].numpy())[:, 0].flatten().tolist()
    batch = {k:v.type(torch.long).cuda() for k,v in batch.items()}

    with torch.no_grad():        
        outputs = model(**batch)

        loss, logits = outputs[:2]
        
        logits = logits.detach().cpu().numpy()
        total_loss += loss.item()

        predict_content = (logits.argmax(axis=-1))[:, 0].flatten().tolist()
        prediction_labels += predict_content

  # Calculate the average loss over the training data.
  dev_avg_epoch_loss = total_loss / len(dev_dataloader)
  #print(len(true_labels))
  #print(len(prediction_labels))
  #print(len(dev_dataloader))
  dev_acc = accuracy_score(true_labels, prediction_labels)

  print("Epoch: %d  train_loss: %.5f  train_acc: %.5f  dev_loss: %.5f  dev_acc: %.5f"%(epoch_i, train_avg_epoch_loss, train_acc, dev_avg_epoch_loss, dev_acc))

ModuleNotFoundError: ignored